# 引入仓库

In [1]:
import geemap
import subprocess
import ee
import os
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
# geemap.set_proxy(port=10809)
ee.Authenticate()
ee.Initialize(project='ee-mrwurenzhe')

# 添加Basemap

In [2]:
# Basemap
Map = geemap.Map(center=[40, -100], zoom=4, add_google_map=False)
Map.add_basemap('HYBRID')
Map.add_basemap('ROADMAP')
# Add Earth Engine data
fc = ee.FeatureCollection('TIGER/2018/Counties')
Map.addLayer(fc, {}, 'US Counties')
states = ee.FeatureCollection('TIGER/2018/States')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# 设计交互式 widgets UI按钮

## 初始Style构建

In [3]:
# Designe interactive widgets

style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

## 设置UI按钮以及初始化值

### Shp输入以及Chart输出定义

In [4]:

# 指定第一个shp，感觉仅为了提供省份的名称
admin1_widget = widgets.Text(
    description='State:', value='Tennessee', width=200, style=style
)
# 指定第二个shp，用于计算方位
admin2_widget = widgets.Text(
    description='County:', value='Knox', width=300, style=style
)
# 使用用户交互式绘图
aoi_widget = widgets.Checkbox(
    value=False, description='Use user-drawn AOI', style=style
)
# 下载chart
download_widget = widgets.Checkbox(
    value=False, description='Download chart data', style=style
)

# 展示button
# widgets.HBox([admin1_widget, admin2_widget, aoi_widget, download_widget])

### 图像按照日期导入/展示/fmask去云和阴影、雪

In [5]:
band_combo = widgets.Dropdown(
    description='Band combo:',
    options=[
        'Red/Green/Blue',
        'NIR/Red/Green',
        'SWIR2/SWIR1/NIR',
        'NIR/SWIR1/Red',
        'SWIR2/NIR/Red',
        'SWIR2/SWIR1/Red',
        'SWIR1/NIR/Blue',
        'NIR/SWIR1/Blue',
        'SWIR2/NIR/Green',
        'SWIR1/NIR/Red',
    ],
    value='NIR/Red/Green',
    style=style,
)

year_widget = widgets.IntSlider(
    min=1984, max=2022, value=2010, description='Selected year:', width=400, style=style
)

fmask_widget = widgets.Checkbox(
    value=True, description='Apply fmask(remove cloud, shadow, snow)', style=style
)
# widgets.HBox([band_combo, year_widget, fmask_widget])

### 波段比值运算选择
 Normalized Satellite Indices: https://www.usna.edu/Users/oceano/pguth/md_help/html/norm_sat.htm

In [6]:
nd_options = [
    'Vegetation Index (NDVI)',
    'Water Index (NDWI)',
    'Modified Water Index (MNDWI)',
    'Snow Index (NDSI)',
    'Soil Index (NDSI)',
    'Burn Ratio (NBR)',
    'Customized',
]
nd_indices = widgets.Dropdown(
    options=nd_options,
    value='Modified Water Index (MNDWI)',
    description='Normalized Difference Index:',
    style=style,
)

first_band = widgets.Dropdown(
    description='1st band:',
    options=['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'],
    value='Green',
    style=style,
)

second_band = widgets.Dropdown(
    description='2nd band:',
    options=['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'],
    value='SWIR1',
    style=style,
)

nd_threshold = widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.01,
    description='Threshold:',
    orientation='horizontal',
    style=style,
)
# 选择颜色
nd_color = widgets.ColorPicker(
    concise=False, description='Color:', value='blue', style=style
)
# widgets.HBox([nd_indices, first_band, second_band, nd_threshold, nd_color])

### 设置Handel

In [7]:
submit = widgets.Button(
    description='Submit', button_style='primary', tooltip='Click me', style=style
)
# widgets.HBox([submit])

### 展示所有的Button

In [8]:
full_widget = widgets.VBox(
    [
        widgets.HBox([admin1_widget, admin2_widget, aoi_widget, download_widget]),
        widgets.HBox([band_combo, year_widget, fmask_widget]),
        widgets.HBox([nd_indices, first_band, second_band, nd_threshold, nd_color]),
        submit,
    ]
)

full_widget

## 计算用户的交互式选择

### 是否采用用户绘制的AOI

In [9]:
# 如果用户选择交互，删除
def aoi_change(change):
    Map.layers = Map.layers[:4]
    Map.user_roi = None
    Map.user_rois = None
    Map.draw_count = 0
    admin1_widget.value = ''
    admin2_widget.value = ''
    output_widget.clear_output()
# 用法类似map
aoi_widget.observe(aoi_change, names='value')

### 波段比值运算
根据选择的Index不同自动更换first_band与second_band的推荐值，并非计算

In [10]:
def nd_index_change(change):
    if nd_indices.value == 'Vegetation Index (NDVI)':
        first_band.value = 'NIR'
        second_band.value = 'Red'
    elif nd_indices.value == 'Water Index (NDWI)':
        first_band.value = 'NIR'
        second_band.value = 'SWIR1'
    elif nd_indices.value == 'Modified Water Index (MNDWI)':
        first_band.value = 'Green'
        second_band.value = 'SWIR1'
    elif nd_indices.value == 'Snow Index (NDSI)':
        first_band.value = 'Green'
        second_band.value = 'SWIR1'
    elif nd_indices.value == 'Soil Index (NDSI)':
        first_band.value = 'SWIR1'
        second_band.value = 'NIR'
    elif nd_indices.value == 'Burn Ratio (NBR)':
        first_band.value = 'NIR'
        second_band.value = 'SWIR2'
    elif nd_indices.value == 'Customized':
        first_band.value = None
        second_band.value = None
nd_indices.observe(nd_index_change, names='value')

### 设置Handel的交互
主要针对采用预制的两个shp进行运算进行参数配置

In [11]:
def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click' and not aoi_widget.value:  # button被点击，并且非用户自定义AOI
        Map.default_style = {'cursor': 'wait'}
        xy = ee.Geometry.Point(latlon[::-1])
        selected_fc = fc.filterBounds(xy)   # fc = ee.FeatureCollection('TIGER/2018/Counties')

        with output_widget:
            output_widget.clear_output()

            try:
                feature = selected_fc.first() # 可能会有多个重叠区域，选择第一个？
                admin2_id = feature.get('NAME').getInfo()
                statefp = feature.get('STATEFP')
                admin1_fc = ee.Feature(
                    states.filter(ee.Filter.eq('STATEFP', statefp)).first()
                )
                admin1_id = admin1_fc.get('NAME').getInfo()
                admin1_widget.value = admin1_id   # 第一个Button，States
                admin2_widget.value = admin2_id   # 第二个Button，citys
                Map.layers = Map.layers[:4]       # Map已经添加了4个layer
                geom = selected_fc.geometry()     # 获取选取的shp切片的地理信息
                layer_name = admin1_id + '-' + admin2_id
                #  # 采用paint调整shp的显示方式geom
                Map.addLayer(
                    ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name
                )
                print(layer_name)
            except:
                print('No feature could be found')
                Map.layers = Map.layers[:4]

        Map.default_style = {'cursor': 'pointer'}
    else:
        Map.draw_count = 0


Map.on_interaction(handle_interaction)


## 根据用户选择计算并展示

In [12]:
# Click event handler
def submit_clicked(b):

    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        try:
            # 两个shp同一个区域的切片导入
            admin1_id = admin1_widget.value
            admin2_id = admin2_widget.value
            # band math采用的波段导入
            band1 = first_band.value
            band2 = second_band.value
            selected_year = year_widget.value    # 年份
            threshold = nd_threshold.value       # 阈值，默认0
            bands = band_combo.value.split('/')  # 波段组合方式
            apply_fmask = fmask_widget.value     # 是否采用fmask
            palette = nd_color.value             # 波段计算颜色选择
            use_aoi = aoi_widget.value           # 人工设置兴趣区域
            download = download_widget.value     # 是否下载
            # 采用人工设置的aoi
            if use_aoi:
                if Map.user_roi is not None:
                    roi = Map.user_roi
                    layer_name = 'User drawn AOI'
                    geom = roi
                else:
                    output_widget.clear_output()
                    print('No user AOI could be found.')
                    return
            # 采用给定shp
            else:
                statefp = ee.Feature(
                    states.filter(ee.Filter.eq('NAME', admin1_id)).first()
                ).get('STATEFP')
                roi = fc.filter(
                    ee.Filter.And(
                        ee.Filter.eq('NAME', admin2_id),
                        ee.Filter.eq('STATEFP', statefp),
                    )
                )
                layer_name = admin1_id + '-' + admin2_id
                geom = roi.geometry()

            Map.layers = Map.layers[:4]          # 导入所有的layers这里总共4个
            Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)
            # 导入图像这里是landsat_timeseries
            images = geemap.landsat_timeseries(
                roi=roi,
                start_year=year_widget.min,
                end_year=year_widget.max,
                start_date='01-01',
                end_date='12-31',
                apply_fmask=apply_fmask,
            )
            # 采用指数进行运算
            nd_images = images.map(lambda img: img.normalizedDifference([band1, band2]))
            result_images = nd_images.map(lambda img: img.gt(threshold))
            # 
            selected_image = ee.Image(
                images.toList(images.size()).get(selected_year - 1984)
            )
            # 指数运算图像,其余区域做掩膜
            selected_result_image = ee.Image(
                result_images.toList(result_images.size()).get(selected_year - 1984)
            ).selfMask()

            vis_params = {'bands': bands, 'min': 0, 'max': 3000}
            # 添加图像和运算后的图层
            Map.addLayer(selected_image, vis_params, 'Landsat ' + str(selected_year))
            Map.addLayer(
                selected_result_image,
                {'palette': palette},
                'Result ' + str(selected_year),
            )
            
            #-----------------------绘制Chart------------------------#
            def cal_area(img):
                pixel_area = img.multiply(ee.Image.pixelArea()).divide(1e4)
                # 在指定区域对geom使用指定的reducer进行运算
                img_area = pixel_area.reduceRegion(
                    **{
                        'geometry': geom,
                        'reducer': ee.Reducer.sum(),
                        'scale': 1000,
                        'maxPixels': 1e12,
                        'bestEffort': True,
                    }
                )
                return img.set({'area': img_area})

            areas = result_images.map(cal_area)
            stats = areas.aggregate_array('area').getInfo()
            x = list(range(1984, 2021))
            y = [item.get('nd') for item in stats]

            fig = plt.figure(1)
            fig.layout.height = '270px'
            plt.clear()
            plt.plot(x, y)
            plt.title('Temporal trend (1984-2020)')
            plt.xlabel('Year')
            plt.ylabel('Area (ha)')

            output_widget.clear_output()

            plt.show()

            if download:
                out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                out_name = 'chart_' + geemap.random_string() + '.csv'
                out_csv = os.path.join(out_dir, out_name)
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)
                with open(out_csv, 'w') as f:
                    f.write('year, area (ha)\n')
                    for index, item in enumerate(x):
                        line = '{},{:.2f}\n'.format(item, y[index])
                        f.write(line)
                link = geemap.create_download_link(
                    out_csv, title="Click here to download the chart data: "
                )
                display(link)

        except Exception as e:
            print(e)
            print('An error occurred during computation.')

        Map.default_style = {'cursor': 'default'}


submit.on_click(submit_clicked)

In [13]:
dir(submit)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_notifiers',
 '_call_widget_constructed',
 '_click_handlers',
 '_comm_changed',
 '_compare',
 '_cross_validation_lock',
 '_default_keys',
 '_display_callbacks',
 '_dom_classes',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_get_trait_default_generator',
 '_handle_button_msg',
 '_handle_custom_msg',
 '_handle_displayed',
 '_handle_msg',
 '_holding_sync',
 '_ipython_display_',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_id',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_observers',
 '_notify_trait',
 '_property_loc